# Implementación de la autoatención con pesos entrenables

En  esta  sección,  implementamos  el  mecanismo  de  autoatención  utilizado  en  la  arquitectura  original  del  transformador,  los  modelos  GPT  y  la  mayoría  de  los  demás  LLM  populares.  

Este  mecanismo  también  se  denomina  atención  escalar  de  producto  escalar.

![Texto alternativo](./imgs/3.12.png)

- La self-attention con pesos entrenables calcula vectores de contexto como combinaciones ponderadas de las entradas.

- La diferencia clave respecto a la versión básica: se introducen matrices de pesos entrenables que se ajustan durante el entrenamiento.

- Estas matrices permiten que el modelo aprenda a producir buenos vectores de contexto.

## Cálculo de los pesos de atención paso a paso

En este apartado se implementa el mecanismo de autoatención introduciendo tres matrices de peso entrenables: `Wq`, `Wk` y `Wv`. Estas matrices se utilizan para proyectar los *tokens* de entrada incrustados `x(i)` en vectores de **consulta (query)**, **clave (key)** y **valor (value)**, respectivamente. 

El procedimiento comienza calculando los vectores `q`, `k` y `v` mediante multiplicaciones de la entrada por las correspondientes matrices de peso. De forma análoga a lo visto en la sección anterior, primero se ilustra el cálculo de un único vector de contexto `z(i)`. Posteriormente, el método se generaliza para obtener todos los vectores de contexto de la secuencia.  

![Texto alternativo](./imgs/3.13.png)

In [1]:
import torch

#Considerar  la  siguiente  oración  de  entrada,  que  ya  ha  sido  incorporada  en  vectores  tridimensionales 
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     
    [0.77, 0.25, 0.10], # one      
    [0.05, 0.80, 0.55]] # step     
)

In [2]:
x_2 = inputs[1] #segundo elemento de la entrada
d_in = inputs.shape[1] #tamaño de incrustación de entrada
d_out = 2 #tamaño de incrustación de salida

#Inicializar tres  matrices  de  peso  Wq ,  Wk  y  Wv

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) #reducir el desorden en las salidas
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

#Calcular los vectores de consulta, clave y valor 
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2) #vector bidimensional


tensor([0.4306, 1.4551])


Los  parámetros  de  peso  son  los  coeficientes  fundamentales  aprendidos  que  definen  las  
conexiones  de  la  red,  mientras  que  los  pesos  de  atención  son  dinámicos  y  específicos  del contexto.

Aunque el objetivo es z_2, z requiere de los vectores de clave y valor, para todos los elemnentos de entradas, ya que están involucrados en el cálculo.

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value
#6 tokens de entrada de un espacion de incrustacion 3D a uno 2D
print("Keys shape: ", keys.shape)
print("Values shape: ", values.shape)


Keys shape:  torch.Size([6, 2])
Values shape:  torch.Size([6, 2])


El segundo paso es calcular los puntuajes de atención

![Texto alternativo](./imgs/3.14.png)

El  cálculo  de  la  puntuación  de  atención  es  un  cálculo  de  producto  escalar  similar  al  utilizado  en  el  mecanismo  simplificado  de  autoatención.  La  novedad  radica  en  que  no  calculamos  directamente  el  producto  escalar  entre  los  elementos  de  entrada,  sino  que  utilizamos  la  consulta  y  la  clave  obtenidas  al  transformar  las  entradas  mediante  las respectivas  matrices  de  ponderación.

Primero calcula el puntuaje de atencio W_22

In [ ]:
keys_2 = keys[1]
query_2 = x_2 @ W_query
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22) #resultado no normalizado

tensor(1.8524)


In [6]:
#generalizar calculo a todos los puntuajes de atención 
attn_score_2 = query_2 @ keys.T
print(attn_score_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


El tercer paso es pasar de los puntuajes de atentión a los pesos de atención

![Texto alternativo](./imgs/3.15.png)

Calcular  las  ponderaciones  de  atención  escalando  las  
puntuaciones  de  atención  y  utilizando  la  función  softmax  que  utilizamos  anteriormente.  La  diferencia  con  la  función  
anterior  radica  en  que  ahora  escalamos  las  puntuaciones  de  atención  dividiéndolas  por  la  raíz  cuadrada  de  la  
dimensión  de  incrustación  de  las  claves  (tenga  en  cuenta  que  calcular  la  raíz  cuadrada  equivale  matemáticamente  a  
exponenciar  por  0,5).